In [1]:
import math
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [119]:
pf = pd.read_csv('EMNA_Comb1.csv')
bm = pd.read_csv('Benchmark_tickers.csv')

In [3]:
# 1. Our portfolio 
pf.head()

,Unnamed: 0,ISSUER_NAME,ISSUERID,ISSUER_TICKER,ISSUER_CUSIP,ISSUER_SEDOL,ISSUER_ISIN,ISSUER_CNTRY_DOMICILE,AS_OF_DATE,IVA_INDUSTRY,...,Country 2,Region,Region_n,yr_n,E_MOM_6M,S_MOM_6M,G_MOM_6M,Rebalance Date,Weights,Portfolio Name
0,151277,Centurytel Inc,IID000000002161445,CTL-N,156700106,2185046,US1567001060,US,11/1/2008,Integrated Telecommunication Services,...,USA,NORTH AMERICA,1,2008,3.061947,0.882353,-0.037778,1/1/2009,0.02,EM_NA_FF
1,111532,Western Digital Corporation,IID000000002189472,WDC,958102105,2954699,US9581021055,US,12/1/2008,Computers & Peripherals,...,USA,NORTH AMERICA,1,2008,2.401042,1.114286,0.639723,1/1/2009,0.02,EM_NA_FF
2,150139,Capital One Financial Corp.,IID000000002133476,COF-N,14040H105,2654461,US14040H1059,US,10/1/2008,Consumer Finance,...,USA,NORTH AMERICA,1,2008,2.363636,-0.320408,-0.545455,1/1/2009,0.02,EM_NA_FF
3,109870,"Avis Budget Group, Inc.",IID000000002149338,CAR-N,53774105,B1CL8J2,US0537741052,US,9/1/2008,Commercial Services & Supplies,...,USA,NORTH AMERICA,1,2008,2.075145,-0.209459,0.021429,1/1/2009,0.02,EM_NA_FF
4,105477,US Bancorp,IID000000002187781,USB,902973304,2736035,US9029733048,US,1/1/2008,Banks - North America,...,USA,NORTH AMERICA,1,2008,1.870270,0.104305,0.637681,1/1/2009,0.02,EM_NA_FF


In [120]:
pf['Year'] = pd.DatetimeIndex(pf['Rebalance Date']).year
pf = pf[['ISSUER_NAME','ISSUER_TICKER','Year','ISSUER_CNTRY_DOMICILE']].dropna()
pf['Ticker'] = pf['ISSUER_TICKER'].apply(lambda x : x.split('-')[0])
pf['Ticker'] = pf['Ticker'].str.replace("'", ".") ##added
pf.rename(columns = {'ISSUER_NAME': 'Name'}, inplace = True)
#pf = pf[pf.ISSUER_CNTRY_DOMICILE == 'US']
pf.head()

,Name,ISSUER_TICKER,Year,ISSUER_CNTRY_DOMICILE,Ticker
0,Centurytel Inc,CTL-N,2009,US,CTL
1,Western Digital Corporation,WDC,2009,US,WDC
2,Capital One Financial Corp.,COF-N,2009,US,COF
3,"Avis Budget Group, Inc.",CAR-N,2009,US,CAR
4,US Bancorp,USB,2009,US,USB


In [121]:
pf['Ticker'][70]

'VIA.B'

In [55]:
# pf has 500 rows but the number of unique tickers is 383!!
len(pf.Ticker) #500
#len(pf.ISSUER_TICKER.unique()) #383

498

In [124]:
pf_tic = pf.Ticker.unique()

In [21]:
#pd.DataFrame(pf_tic).to_csv('pf_tic.csv')

In [74]:
len(pf_tic) ###373 tickers###

373

In [8]:
# 2. Benchmark : Equity in iShares-MSCI-ACWI-ETF_fund
bm.head()

,Ticker,Name,Asset Class
0,AAPL,APPLE INC,Equity
1,MSFT,MICROSOFT CORP,Equity
2,AMZN,AMAZON COM INC,Equity
3,INDA,ISHARES MSCI INDIA ETF,Equity
4,JNJ,JOHNSON & JOHNSON,Equity


In [123]:
bm = bm[['Ticker', 'Name', 'Asset Class']].drop_duplicates()
bm = bm[bm['Asset Class'] == 'Equity']
bm = bm.dropna(subset = ['Ticker'])
bm.head()

,Ticker,Name,Asset Class
0,AAPL,APPLE INC,Equity
1,MSFT,MICROSOFT CORP,Equity
2,AMZN,AMAZON COM INC,Equity
3,INDA,ISHARES MSCI INDIA ETF,Equity
4,JNJ,JOHNSON & JOHNSON,Equity


In [40]:
len(bm.Ticker) # 1370
len(bm.Ticker.unique()) # 1337 unique tickers same!

1337

In [115]:
bm.shape # 1370 rows

(1370, 3)

In [125]:
bm_tic = bm.Ticker.unique()
bm_tic

array(['AAPL', 'MSFT', 'AMZN', ..., '6863', '012630', 'BES'], dtype=object)

In [126]:
pf['Ticker'].isin(bm['Ticker']).sum() # 186 common stocks

186

In [127]:
pfbm_tic = np.concatenate((pf_tic, bm_tic), axis = None)
len(pfbm_tic) # 1710      # 373 + 1337 = 1710

1709

In [90]:
#pd.DataFrame(pfbm_tic).to_csv('pfbm_tic_new.csv')

#### For each year, modify here

In [214]:
# 3. Tone
YEAR = 2018
pf_oneYear = pf[pf.Year == YEAR]
sdg_data = pd.read_csv('sdgtone_18.csv')

In [215]:
sdg_data.head()

,Unnamed: 0,date,code,SDG,tone
0,0,20180101,--,1. No Poverty,-0.586243
1,1,20180101,--,10. Reduced Inequalities,-0.495595
2,2,20180101,--,11. Sustainable Cities and Communities,-0.794117
3,3,20180101,--,12. Responsible Consumption and Production,-0.089141
4,4,20180101,--,13. Climate Action,0.265076


In [216]:
type(sdg_data.date[0])

numpy.int64

In [212]:
sdg_data_tic = sdg_data.Ticker.unique()

In [217]:
len(sdg_data_tic) # 521   #7808

521

In [219]:
sdg_subset = sdg_data.loc[sdg_data['code'].isin(pfbm_tic)]

In [220]:
sdg_subset.tail() # until 20181125

,Unnamed: 0,date,code,SDG,tone
29950023,29950023,20181125,ZURN,"16. Peace, Justice and Strong Institutions",0.311085
29950024,29950024,20181125,ZURN,3. Good Health and Well-being,0.201816
29950025,29950025,20181125,ZURN,6. Clean Water and Sanitaion,0.201816
29950026,29950026,20181125,ZURN,7. Affordable and Clean Energy,0.201816
29950027,29950027,20181125,ZURN,8. Decent Work and Economic Growth,0.494438


In [221]:
len(sdg_subset.code.unique()) # 521 ## 1538

1538

In [134]:
str(sdg_subset['date'][0])

'2018-01-01'

In [223]:
#Convert date from string to datetime
sdg_subset['date'] = sdg_subset['date'].apply(lambda x: str(x)[:10])
sdg_subset['date'] = sdg_subset['date'].apply(lambda x: datetime.strptime(x, '%Y%m%d'))

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [177]:
sdg_subset.head()

,Unnamed: 0,Ticker,SDG_number,date,tone,tone_agg
0,0,A,1,2018-01-01,2.397260,NaN
1,1,A,1,2018-01-02,-2.521460,NaN
2,2,A,1,2018-01-03,-4.749159,NaN
3,3,A,1,2018-01-04,-0.904159,NaN
4,4,A,1,2018-01-05,2.568883,NaN


In [178]:
## Staleness_Overall_2018 ##
sdg_subset_overall = sdg_subset.groupby(['Ticker', 'date'], as_index = False)['tone'].mean()
sdg_subset_overall.head()

,Ticker,date,tone
0,A,2018-01-01,0.550159
1,A,2018-01-02,-2.012712
2,A,2018-01-03,-1.509756
3,A,2018-01-04,0.621842
4,A,2018-01-05,0.839174


In [139]:
##########Mktcap new dataset from CRSP
mktcap = pd.read_csv('MktCap_bmpf_new_crsp.csv')

In [140]:
mktcap = mktcap.drop_duplicates(subset=['TICKER'], keep = 'last')
mktcap.head()

,PERMNO,date,TICKER,COMNAM,CUSIP,PRC,SHROUT
23,10104,20181231,ORCL,ORACLE CORP,68389X10,45.15,3588919.0
47,10107,20181231,MSFT,MICROSOFT CORP,59491810,101.57,7676219.0
71,10138,20181231,TROW,T ROWE PRICE GROUP INC,74144T10,92.32,240655.0
95,10145,20181231,HON,HONEYWELL INTERNATIONAL INC,43851610,132.12,740288.0
114,10239,20180731,BWINB,BALDWIN & LYONS INC,74368L20,23.35,12380.0


In [142]:
mktcap['mktcap'] = abs(mktcap['PRC']*mktcap['SHROUT'])
mktcap.head()

,PERMNO,date,TICKER,COMNAM,CUSIP,PRC,SHROUT,mktcap
23,10104,20181231,ORCL,ORACLE CORP,68389X10,45.15,3588919.0,1.620397e+08
47,10107,20181231,MSFT,MICROSOFT CORP,59491810,101.57,7676219.0,7.796736e+08
71,10138,20181231,TROW,T ROWE PRICE GROUP INC,74144T10,92.32,240655.0,2.221727e+07
95,10145,20181231,HON,HONEYWELL INTERNATIONAL INC,43851610,132.12,740288.0,9.780685e+07
114,10239,20180731,BWINB,BALDWIN & LYONS INC,74368L20,23.35,12380.0,2.890730e+05


In [143]:
len(mktcap.TICKER.unique())  #922 tickers in the Mktcap

922

In [179]:
mktcap_merged = pd.merge(sdg_subset_overall, mktcap, left_on = ['Ticker'], right_on = ['TICKER'])
mktcap_merged.head()

,Ticker,date_x,tone,PERMNO,date_y,TICKER,COMNAM,CUSIP,PRC,SHROUT,mktcap
0,A,2018-01-01,0.550159,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18
1,A,2018-01-02,-2.012712,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18
2,A,2018-01-03,-1.509756,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18
3,A,2018-01-04,0.621842,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18
4,A,2018-01-05,0.839174,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18


In [181]:
len(mktcap_merged.Ticker.unique())

518

In [196]:
mktcap_merged['size_dollar'] = mktcap_merged['mktcap']*1000
mktcap_merged.head()

,Ticker,date_x,tone,PERMNO,date_y,TICKER,COMNAM,CUSIP,PRC,SHROUT,mktcap,Size,size_dollar
0,A,2018-01-01,0.550159,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18,Medium,2.148824e+10
1,A,2018-01-02,-2.012712,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18,Medium,2.148824e+10
2,A,2018-01-03,-1.509756,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18,Medium,2.148824e+10
3,A,2018-01-04,0.621842,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18,Medium,2.148824e+10
4,A,2018-01-05,0.839174,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18,Medium,2.148824e+10


In [197]:
mktcap_merged.size_dollar.describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99])

count    1.691060e+05
mean     4.171108e+10
std      7.713529e+10
min      4.254073e+07
10%      4.927052e+09
20%      8.677812e+09
30%      1.103786e+10
40%      1.428948e+10
50%      1.830479e+10
60%      2.339832e+10
70%      3.394394e+10
80%      4.996366e+10
90%      9.077131e+10
95%      1.831429e+11
99%      3.246266e+11
max      7.796736e+11
Name: size_dollar, dtype: float64

In [194]:
mktcap_merged.mktcap.describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99])

count    1.691060e+05
mean     4.171108e+07
std      7.713529e+07
min      4.254073e+04
10%      4.927052e+06
20%      8.677812e+06
30%      1.103786e+07
40%      1.428948e+07
50%      1.830479e+07
60%      2.339832e+07
70%      3.394394e+07
80%      4.996366e+07
90%      9.077131e+07
95%      1.831429e+08
99%      3.246266e+08
max      7.796736e+08
Name: mktcap, dtype: float64

In [182]:
upperlimit = mktcap_merged.mktcap.describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])['80%']
lowerlimit = mktcap_merged.mktcap.describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])['50%']

In [183]:
# A function to assign a size bucket
def label_group (row):
    if row['mktcap'] < lowerlimit:
        return 'Small'
    if (row['mktcap'] >= lowerlimit) & (row['mktcap'] < upperlimit):
        return 'Medium'
    if row['mktcap'] > upperlimit:
        return 'Large'

In [184]:
mktcap_merged['Size'] = mktcap_merged.apply (lambda row: label_group (row),axis=1)
mktcap_merged.head()

,Ticker,date_x,tone,PERMNO,date_y,TICKER,COMNAM,CUSIP,PRC,SHROUT,mktcap,Size
0,A,2018-01-01,0.550159,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18,Medium
1,A,2018-01-02,-2.012712,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18,Medium
2,A,2018-01-03,-1.509756,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18,Medium
3,A,2018-01-04,0.621842,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18,Medium
4,A,2018-01-05,0.839174,87432,20181231,A,AGILENT TECHNOLOGIES INC,00846U10,67.46,318533.0,21488236.18,Medium


In [185]:
mktcap_merged_subset = mktcap_merged[['Ticker', 'date_x', 'tone', 'mktcap', 'Size']]
mktcap_merged_subset.rename(columns = {'date_x': 'date'}, inplace = True)
mktcap_merged_subset.head()

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Ticker,date,tone,mktcap,Size
0,A,2018-01-01,0.550159,21488236.18,Medium
1,A,2018-01-02,-2.012712,21488236.18,Medium
2,A,2018-01-03,-1.509756,21488236.18,Medium
3,A,2018-01-04,0.621842,21488236.18,Medium
4,A,2018-01-05,0.839174,21488236.18,Medium


In [186]:
mktcap_merged_bySize = [] 

for size in ['Small', 'Medium', 'Large']:
    subset_bySize = mktcap_merged_subset[mktcap_merged_subset['Size'] == size]
    mktcap_merged_bySize.append(subset_bySize)

small = mktcap_merged_bySize[0]
medium = mktcap_merged_bySize[1]
large = mktcap_merged_bySize[2]

large.head()

,Ticker,date,tone,mktcap,Size
658,AAPL,2018-01-01,0.113991,7.485391e+08,Large
659,AAPL,2018-01-02,-1.101240,7.485391e+08,Large
660,AAPL,2018-01-03,-1.569102,7.485391e+08,Large
661,AAPL,2018-01-04,-1.343982,7.485391e+08,Large
662,AAPL,2018-01-05,-1.162373,7.485391e+08,Large


In [187]:
len(small.Ticker.unique()) # 257

257

In [188]:
len(medium.Ticker.unique()) # 154

154

In [189]:
len(large.Ticker.unique()) #102

102

In [ ]:
############################################

In [190]:
def stale_eachSDG(x):
    
    df_pivot = pd.pivot_table(x, values='tone', index=['date'], columns=['Ticker']).reset_index()
    date_range_all = pd.DataFrame(pd.date_range(df_pivot.date[0], df_pivot.date.tolist()[-1], freq='D'), columns=['date'])
    df_all_dates = date_range_all.merge(df_pivot, on='date', how = 'left')
    df_all_dates['date'] = df_all_dates['date'].apply(lambda x: str(x)[:10])

    # Run the function
    df_staleness_short = pd.DataFrame()

    for col in df_all_dates.columns[1:]:
    df_staleness_short[col] = df_all_dates[col].rolling(7).count()

    df_staleness_short['date'] = df_all_dates.date
    df_staleness_short = df_staleness_short.set_index('date')

    #Summary Statistic
    df_staleness_short['avg'] = df_staleness_short.mean(axis = 1)
    avg_day = df_staleness_short['avg'].describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.95, 0.975, 0.99])

    df_staleness_short.loc['avg_tic'] = df_staleness_short.mean()
    avg_tic = df_staleness_short.loc['avg_tic'].describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.975, 0.99])

    return avg_day, avg_tic

In [191]:
stale_eachSDG(small)

(count    329.000000
 mean       6.757939
 std        0.520447
 min        0.918288
 10%        6.731518
 20%        6.772763
 30%        6.805447
 40%        6.817121
 50%        6.828794
 60%        6.843580
 70%        6.856031
 80%        6.867704
 95%        6.894942
 97.5%      6.906615
 99%        6.924981
 max        6.929961
 Name: avg, dtype: float64, count    258.000000
 mean       6.757939
 std        0.652416
 min        0.127660
 10%        6.465046
 20%        6.765957
 30%        6.851064
 40%        6.893617
 50%        6.914894
 60%        6.936170
 70%        6.936170
 80%        6.936170
 90%        6.936170
 95%        6.936170
 97.5%      6.936170
 99%        6.936170
 max        6.936170
 Name: avg_tic, dtype: float64)

In [192]:
stale_eachSDG(medium)

(count    329.000000
 mean       6.850709
 std        0.519302
 min        0.967532
 10%        6.857143
 20%        6.883117
 30%        6.896104
 40%        6.909091
 50%        6.915584
 60%        6.928571
 70%        6.935065
 80%        6.948052
 95%        6.974026
 97.5%      6.980519
 99%        6.987013
 max        6.993506
 Name: avg, dtype: float64, count    155.000000
 mean       6.850709
 std        0.287666
 min        3.814590
 10%        6.744681
 20%        6.851064
 30%        6.893617
 40%        6.914894
 50%        6.936170
 60%        6.936170
 70%        6.936170
 80%        6.936170
 90%        6.936170
 95%        6.936170
 97.5%      6.936170
 99%        6.936170
 max        6.936170
 Name: avg_tic, dtype: float64)

In [193]:
stale_eachSDG(large)

(count    329.000000
 mean       6.856308
 std        0.518164
 min        0.980392
 10%        6.872549
 20%        6.882353
 30%        6.901961
 40%        6.911765
 50%        6.921569
 60%        6.931373
 70%        6.941176
 80%        6.950980
 95%        6.960784
 97.5%      6.970588
 99%        6.970588
 max        6.990196
 Name: avg, dtype: float64, count    103.000000
 mean       6.856308
 std        0.320615
 min        4.446809
 10%        6.834043
 20%        6.914894
 30%        6.936170
 40%        6.936170
 50%        6.936170
 60%        6.936170
 70%        6.936170
 80%        6.936170
 90%        6.936170
 95%        6.936170
 97.5%      6.936170
 99%        6.936170
 max        6.936170
 Name: avg_tic, dtype: float64)

In [86]:
df_pivot = pd.pivot_table(t18_subset_overall, values='tone', index=['date'], columns=['code']).reset_index()
df_pivot.head()

code,date,--,000002,000030,000270,000333,000660,000810,000858,000880,...,YUMC,YY,Z,Z74,ZAL,ZBH,ZEUS,ZG,ZTS,ZURN
0,2018-01-01,-0.275787,-0.542697,-1.481917,-0.068438,-0.312685,-2.118511,-2.185846,0.000000,NaN,...,-0.804376,2.098500,-3.973051,-0.839335,NaN,-0.646168,-2.215477,-3.992301,0.902643,-0.195780
1,2018-01-02,1.120522,-2.140108,0.785727,-0.822067,-1.716658,0.439787,-0.360387,-2.679957,NaN,...,-0.668816,-2.485262,0.594807,1.018639,NaN,0.653595,0.439787,-1.447627,-2.531392,-1.951179
2,2018-01-03,-0.321878,-3.225213,0.464779,0.003496,-0.622466,-0.602333,-0.428390,-5.511031,NaN,...,-1.766175,-2.972815,-1.377619,-0.484250,NaN,-0.864598,0.855952,-1.390290,-5.082383,-2.775698
3,2018-01-04,-0.147493,0.576295,-0.969910,-0.214815,-0.166813,-0.675318,-0.671540,2.516041,2.217742,...,1.014276,1.871424,-0.542270,0.403237,NaN,0.284156,0.507001,-0.950149,1.252119,-0.482036
4,2018-01-05,1.022425,-0.130876,0.493381,0.708416,0.637241,-0.560832,-0.702478,3.449198,NaN,...,-0.009154,2.003963,0.776566,1.138212,NaN,1.577471,-0.033127,0.625949,1.651955,0.420657


In [87]:
date_range_all = pd.DataFrame(pd.date_range(df_pivot.date[0], df_pivot.date.tolist()[-1], freq='D'), columns=['date'])
date_range_all.head()

,date
0,2018-01-01
1,2018-01-02
2,2018-01-03
3,2018-01-04
4,2018-01-05


In [88]:
df_all_dates = date_range_all.merge(df_pivot, on='date', how = 'left')

In [89]:
df_all_dates.head() 

,date,--,000002,000030,000270,000333,000660,000810,000858,000880,...,YUMC,YY,Z,Z74,ZAL,ZBH,ZEUS,ZG,ZTS,ZURN
0,2018-01-01,-0.275787,-0.542697,-1.481917,-0.068438,-0.312685,-2.118511,-2.185846,0.000000,NaN,...,-0.804376,2.098500,-3.973051,-0.839335,NaN,-0.646168,-2.215477,-3.992301,0.902643,-0.195780
1,2018-01-02,1.120522,-2.140108,0.785727,-0.822067,-1.716658,0.439787,-0.360387,-2.679957,NaN,...,-0.668816,-2.485262,0.594807,1.018639,NaN,0.653595,0.439787,-1.447627,-2.531392,-1.951179
2,2018-01-03,-0.321878,-3.225213,0.464779,0.003496,-0.622466,-0.602333,-0.428390,-5.511031,NaN,...,-1.766175,-2.972815,-1.377619,-0.484250,NaN,-0.864598,0.855952,-1.390290,-5.082383,-2.775698
3,2018-01-04,-0.147493,0.576295,-0.969910,-0.214815,-0.166813,-0.675318,-0.671540,2.516041,2.217742,...,1.014276,1.871424,-0.542270,0.403237,NaN,0.284156,0.507001,-0.950149,1.252119,-0.482036
4,2018-01-05,1.022425,-0.130876,0.493381,0.708416,0.637241,-0.560832,-0.702478,3.449198,NaN,...,-0.009154,2.003963,0.776566,1.138212,NaN,1.577471,-0.033127,0.625949,1.651955,0.420657


In [90]:
df_all_dates['date'] = df_all_dates['date'].apply(lambda x: str(x)[:10])

In [91]:
df_all_dates.head()  ########Final df########

,date,--,000002,000030,000270,000333,000660,000810,000858,000880,...,YUMC,YY,Z,Z74,ZAL,ZBH,ZEUS,ZG,ZTS,ZURN
0,2018-01-01,-0.275787,-0.542697,-1.481917,-0.068438,-0.312685,-2.118511,-2.185846,0.000000,NaN,...,-0.804376,2.098500,-3.973051,-0.839335,NaN,-0.646168,-2.215477,-3.992301,0.902643,-0.195780
1,2018-01-02,1.120522,-2.140108,0.785727,-0.822067,-1.716658,0.439787,-0.360387,-2.679957,NaN,...,-0.668816,-2.485262,0.594807,1.018639,NaN,0.653595,0.439787,-1.447627,-2.531392,-1.951179
2,2018-01-03,-0.321878,-3.225213,0.464779,0.003496,-0.622466,-0.602333,-0.428390,-5.511031,NaN,...,-1.766175,-2.972815,-1.377619,-0.484250,NaN,-0.864598,0.855952,-1.390290,-5.082383,-2.775698
3,2018-01-04,-0.147493,0.576295,-0.969910,-0.214815,-0.166813,-0.675318,-0.671540,2.516041,2.217742,...,1.014276,1.871424,-0.542270,0.403237,NaN,0.284156,0.507001,-0.950149,1.252119,-0.482036
4,2018-01-05,1.022425,-0.130876,0.493381,0.708416,0.637241,-0.560832,-0.702478,3.449198,NaN,...,-0.009154,2.003963,0.776566,1.138212,NaN,1.577471,-0.033127,0.625949,1.651955,0.420657


In [92]:
len(df_all_dates)

329

In [93]:
df_staleness_short = pd.DataFrame()
for col in df_all_dates.columns[1:]:
    df_staleness_short[col] = df_all_dates[col].rolling(7).count()
df_staleness_short['date'] = df_all_dates.date
df_staleness_short = df_staleness_short.set_index('date')
df_staleness_short.head(10)

,--,000002,000030,000270,000333,000660,000810,000858,000880,001040,...,YUMC,YY,Z,Z74,ZAL,ZBH,ZEUS,ZG,ZTS,ZURN
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2018-01-02,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,1.0,...,2.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0
2018-01-03,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,2.0,...,3.0,3.0,3.0,3.0,0.0,3.0,3.0,3.0,3.0,3.0
2018-01-04,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.0,...,4.0,4.0,4.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0
2018-01-05,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,4.0,...,5.0,5.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0
2018-01-06,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,2.0,4.0,...,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,6.0,6.0
2018-01-07,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,3.0,5.0,...,7.0,7.0,7.0,7.0,0.0,7.0,7.0,7.0,7.0,7.0
2018-01-08,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,4.0,6.0,...,7.0,7.0,7.0,7.0,0.0,7.0,7.0,7.0,7.0,7.0
2018-01-09,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,5.0,6.0,...,7.0,7.0,7.0,7.0,0.0,7.0,7.0,7.0,7.0,7.0


In [94]:
len(df_staleness_short)

329

In [95]:
df_staleness_short['avg'] = df_staleness_short.mean(axis = 1)

In [96]:
df_staleness_short.head()

,--,000002,000030,000270,000333,000660,000810,000858,000880,001040,...,YY,Z,Z74,ZAL,ZBH,ZEUS,ZG,ZTS,ZURN,avg
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.906352
2018-01-02,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,1.0,...,2.0,2.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,1.839555
2018-01-03,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,2.0,...,3.0,3.0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,2.808775
2018-01-04,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.0,...,4.0,4.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,3.787819
2018-01-05,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,4.0,...,5.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0,4.769483


In [97]:
 df_staleness_short['avg'].describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

count    329.000000
mean       6.645995
std        0.510714
min        0.906352
10%        6.629862
20%        6.665357
30%        6.689587
40%        6.705435
50%        6.716437
60%        6.725606
70%        6.735167
80%        6.753111
90%        6.766470
max        6.806811
Name: avg, dtype: float64

In [98]:
df_staleness_short.loc['avg_tic'] = df_staleness_short.mean()

In [99]:
df_staleness_short.tail(10)

,--,000002,000030,000270,000333,000660,000810,000858,000880,001040,...,YY,Z,Z74,ZAL,ZBH,ZEUS,ZG,ZTS,ZURN,avg
date,,,,,,,,,,,,,,,,,,,,,
2018-11-17,7.00000,7.000000,7.000000,7.00000,7.00000,7.000000,7.00000,7.000000,2.000000,7.00000,...,7.000000,7.000000,7.00000,0.000000,7.000000,7.000000,7.000000,7.00000,7.000000,6.748527
2018-11-18,7.00000,7.000000,7.000000,7.00000,7.00000,7.000000,7.00000,6.000000,2.000000,7.00000,...,7.000000,7.000000,7.00000,0.000000,7.000000,7.000000,7.000000,7.00000,7.000000,6.716437
2018-11-19,7.00000,7.000000,7.000000,7.00000,7.00000,7.000000,7.00000,6.000000,2.000000,7.00000,...,7.000000,7.000000,7.00000,0.000000,7.000000,7.000000,7.000000,7.00000,7.000000,6.747217
2018-11-20,7.00000,7.000000,7.000000,7.00000,7.00000,7.000000,7.00000,6.000000,3.000000,6.00000,...,7.000000,7.000000,7.00000,0.000000,7.000000,7.000000,7.000000,7.00000,7.000000,6.726261
2018-11-21,7.00000,7.000000,7.000000,7.00000,7.00000,7.000000,7.00000,6.000000,3.000000,6.00000,...,7.000000,7.000000,7.00000,0.000000,7.000000,7.000000,7.000000,7.00000,7.000000,6.732155
2018-11-22,7.00000,7.000000,7.000000,7.00000,7.00000,7.000000,7.00000,6.000000,2.000000,6.00000,...,7.000000,7.000000,7.00000,0.000000,7.000000,7.000000,7.000000,7.00000,7.000000,6.729535
2018-11-23,7.00000,7.000000,7.000000,7.00000,7.00000,7.000000,7.00000,6.000000,1.000000,6.00000,...,7.000000,7.000000,7.00000,0.000000,7.000000,7.000000,7.000000,7.00000,7.000000,6.716437
2018-11-24,7.00000,7.000000,6.000000,7.00000,7.00000,7.000000,7.00000,6.000000,1.000000,6.00000,...,7.000000,7.000000,7.00000,0.000000,7.000000,7.000000,7.000000,7.00000,7.000000,6.719712
2018-11-25,7.00000,7.000000,6.000000,7.00000,7.00000,7.000000,7.00000,6.000000,1.000000,6.00000,...,7.000000,7.000000,7.00000,0.000000,7.000000,7.000000,6.000000,7.00000,7.000000,6.656843


In [100]:
df_staleness_short.loc['avg_tic'].describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.97, 0.99])

count    1528.000000
mean        6.645995
std         0.859004
min         0.021277
10%         6.255319
20%         6.659574
30%         6.829787
40%         6.893617
50%         6.914894
60%         6.936170
70%         6.936170
80%         6.936170
90%         6.936170
95%         6.936170
97%         6.936170
99%         6.936170
max         6.936170
Name: avg_tic, dtype: float64

In [101]:
##For each 17 SDG##
t18_subset.head()

,Unnamed: 0,date,code,SDG,tone
0,0,2018-01-01,--,1. No Poverty,-0.586243
1,1,2018-01-01,--,10. Reduced Inequalities,-0.495595
2,2,2018-01-01,--,11. Sustainable Cities and Communities,-0.794117
3,3,2018-01-01,--,12. Responsible Consumption and Production,-0.089141
4,4,2018-01-01,--,13. Climate Action,0.265076


In [102]:
# Create SDG_number column
def getSDG(str):
    sdg = int(str.split('.')[0])
    return sdg

t18_subset['SDG_number'] = t18_subset['SDG'].apply(lambda x: getSDG(x))
t18_subset.head()

/Users/Sophshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Unnamed: 0,date,code,SDG,tone,SDG_number
0,0,2018-01-01,--,1. No Poverty,-0.586243,1
1,1,2018-01-01,--,10. Reduced Inequalities,-0.495595,10
2,2,2018-01-01,--,11. Sustainable Cities and Communities,-0.794117,11
3,3,2018-01-01,--,12. Responsible Consumption and Production,-0.089141,12
4,4,2018-01-01,--,13. Climate Action,0.265076,13


In [ ]:
t18_subset['SDG_number'] = t18_subset['SDG_number'].apply(lambda x: int(x))

In [ ]:
###########new#######################

In [ ]:
t18_subset_bySDG = [] 

for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]:
    subset_bySDG = t18_subset.loc[t18_subset['SDG_number'] == i]
    t18_subset_bySDG.append(subset_bySDG)

In [ ]:
t18_subset_bySDG[0].head()

In [ ]:
def stale_eachSDG(x):
    
    df_pivot = pd.pivot_table(x, values='tone', index=['date'], columns=['code']).reset_index()
    date_range_all = pd.DataFrame(pd.date_range(df_pivot.date[0], df_pivot.date.tolist()[-1], freq='D'), columns=['date'])
    df_all_dates = date_range_all.merge(df_pivot, on='date', how = 'left')
    df_all_dates['date'] = df_all_dates['date'].apply(lambda x: str(x)[:10])
  
    # Run the function
    df_staleness_short = pd.DataFrame()
  
    for col in df_all_dates.columns[1:]:
        df_staleness_short[col] = df_all_dates[col].rolling(7).count()
        
        df_staleness_short['date'] = df_all_dates.date
        df_staleness_short = df_staleness_short.set_index('date')

        #Summary Statistic
        df_staleness_short['avg'] = df_staleness_short.mean(axis = 1)
        avg_day = df_staleness_short['avg'].describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.95, 0.975, 0.99])

        df_staleness_short.loc['avg_tic'] = df_staleness_short.mean()
        avg_tic = df_staleness_short.loc['avg_tic'].describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.975, 0.99])

    return avg_day, avg_tic


In [ ]:
#mannually change 0 to 16
stale_eachSDG(t18_subset_bySDG[i]

In [ ]:
####might be confusing
stat_final = []

for i in range(17):
    summary = stale_eachSDG(t18_subset_bySDG[i])
    stat_final.append(summary)

In [ ]:
##################old___ignore below##################################

In [103]:
df_staleness_short_bySDG = t18_subset.set_index('date')
df_staleness_short_bySDG = df_staleness_short_bySDG.groupby(['code', 'SDG_number'])['tone'].rolling(7).count().reset_index()
df_staleness_short_bySDG.head(10)

,code,SDG_number,date,tone
0,--,1,2018-01-01,1.0
1,--,1,2018-01-02,2.0
2,--,1,2018-01-03,3.0
3,--,1,2018-01-04,4.0
4,--,1,2018-01-05,5.0
5,--,1,2018-01-06,6.0
6,--,1,2018-01-07,7.0
7,--,1,2018-01-08,7.0
8,--,1,2018-01-09,7.0
9,--,1,2018-01-10,7.0


In [114]:
# Averaged over the companies for each day
df_staleness_short_bySDG_aveCount = df_staleness_short_bySDG.groupby(['SDG_number', 'date'])['tone'].mean().reset_index()
df_staleness_short_bySDG_aveCount.head(10)

,SDG_number,date,tone
0,1,2018-01-01,1.000000
1,1,2018-01-02,1.342266
2,1,2018-01-03,1.872861
3,1,2018-01-04,2.792220
4,1,2018-01-05,3.722222
5,1,2018-01-06,4.650320
6,1,2018-01-07,5.649425
7,1,2018-01-08,4.678881
8,1,2018-01-09,5.202776
9,1,2018-01-10,5.864341


In [115]:
grouped_bySDG = df_staleness_short_bySDG_aveCount.groupby(['SDG_number'])
for name, group in grouped_bySDG:
    print ('SDG_'+ str(name) + ' Averaged over the companies for each day')
    print (group['tone'].describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.95, 0.975, 0.99]))

SDG_1 Averaged over the companies for each day
count    329.000000
mean       6.886906
std        0.646255
min        1.000000
10%        6.976181
20%        6.993245
30%        6.997756
40%        7.000000
50%        7.000000
60%        7.000000
70%        7.000000
80%        7.000000
95%        7.000000
97.5%      7.000000
99%        7.000000
max        7.000000
Name: tone, dtype: float64
SDG_2 Averaged over the companies for each day
count    329.000000
mean       6.905481
std        0.588640
min        1.000000
10%        6.986320
20%        6.994595
30%        6.997545
40%        7.000000
50%        7.000000
60%        7.000000
70%        7.000000
80%        7.000000
95%        7.000000
97.5%      7.000000
99%        7.000000
max        7.000000
Name: tone, dtype: float64
SDG_3 Averaged over the companies for each day
count    329.000000
mean       6.927495
std        0.548022
min        1.000000
10%        6.995935
20%        6.998606
30%        7.000000
40%        7.000000
50%  

In [106]:
#Averaged over days for each company
df_staleness_short_bySDG_aveCount = df_staleness_short_bySDG.groupby(['SDG_number', 'code'])['tone'].mean().reset_index()
df_staleness_short_bySDG_aveCount.head(10)

,SDG_number,code,tone
0,1,--,6.932039
1,1,000002,6.902326
2,1,000030,6.888298
3,1,000270,6.928082
4,1,000333,6.906667
5,1,000660,6.860927
6,1,000810,6.935780
7,1,000858,6.800000
8,1,000880,6.000000
9,1,001040,6.686567


In [108]:
grouped_bySDG = df_staleness_short_bySDG_aveCount.groupby(['SDG_number'])
for name, group in grouped_bySDG:
    print (name)
    print (group['tone'].describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.95, 0.975, 0.99]))

1
count    1522.000000
mean        6.793524
std         0.497686
min         1.000000
10%         6.727273
20%         6.801887
30%         6.844444
40%         6.874251
50%         6.889474
60%         6.900000
70%         6.909091
80%         6.918605
95%         6.933962
97.5%       6.935976
99%         6.936170
max         6.936170
Name: tone, dtype: float64
2
count    1522.000000
mean        6.832808
std         0.472143
min         1.000000
10%         6.807339
20%         6.860927
30%         6.888298
40%         6.901869
50%         6.909483
60%         6.916667
70%         6.921933
80%         6.927336
95%         6.935185
97.5%       6.936170
99%         6.936170
max         6.936170
Name: tone, dtype: float64
3
count    1527.000000
mean        6.897975
std         0.348359
min         1.000000
10%         6.913934
20%         6.924785
30%         6.930233
40%         6.932907
50%         6.934375
60%         6.935385
70%         6.935780
80%         6.936170
95%         6.93